---

# Simplified access to Veluwe ecological data through OGC Web Services

**Author:** Hudson Passos  
**Internship host:** Netherlands Institute of Ecology (NIOO-KNAW)  
**Host supervisor:** Stefan Vriend (NIOO-KNAW)  
**WUR supervisor:** Liesbeth Bakker (WUR, NIOO-KNAW)  
**Repository:** [research-project-internship-nioo](https://github.com/hudsonpassos/research-project-internship-nioo)  
**Date:** July 18, 2025  
**Python version:** 3.11.9  
**License:** MIT  
**Description:**  
This notebook is part of a research internship project. It focuses on the automated selection, filtering, 
and preprocessing of open ecological geospatial datasets for the Veluwe region using OGC Web Services (WCS and WFS).


---

# **Part 4**: Download via WCS and WFS

### 1.1. Initialization: packages, paths, and spatial inputs

**Packages**

In [ ]:
import pandas as pd
import geopandas as gpd
import time
from PIL import Image
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from owslib.csw import CatalogueServiceWeb
from owslib.ows import ServiceIdentification
import requests
from lxml import etree
import pandas as pd
import re
import ast
from pathlib import Path
from urllib.parse import urlencode
from xml.etree import ElementTree as ET
from shapely.geometry import box
from urllib.parse import urlparse, urlunparse
from urllib.parse import urlparse, urlunparse
from rasterio.merge import merge
import rasterio
import glob
import math
from pyproj import Transformer
from urllib.parse import urlencode
import tempfile
#import xml.etree.ElementTree as ET
from urllib.parse import urlparse, urlunparse, urlencode
import subprocess
from osgeo import gdal

**Setting pathways**

In [ ]:
root = "C:/Users/hudso/OneDrive/Documents/02. WUR/11. INTERNSHIP"
outlines_path = os.path.join(root, "02 data/outlines")
output_path = os.path.join(root, "05 python_project/output")

**Loading data frames and creating column 'download'**

In [ ]:
df_wcs = pd.read_csv("checkpoint03_sf2_ngr_WCS_metadata.csv")
df_wfs = pd.read_csv("checkpoint04_ngr_WFS_metadata.csv")

df_wcs['download'] = 'no'
df_wfs['download'] = 'no'

df_wcs.to_csv("checkpoint05_ngr_WCS_metadata.csv", index=False)
df_wfs.to_csv("checkpoint05_ngr_WFS_metadata.csv", index=False)

**Loading outlines**

In [ ]:
NLshp_path = os.path.join(outlines_path, "netherlands28992_outline.gpkg")
VELshp_path = os.path.join(outlines_path, "veluwe28992_outline.gpkg")

NLgdf = gpd.read_file(NLshp_path)
VELgdf = gpd.read_file(VELshp_path)

netherlands4326 = NLgdf.to_crs(epsg=4326)
netherlands28992 = NLgdf

veluwe4326 = VELgdf.to_crs(epsg=4326)
veluwe28992 = VELgdf

**Function to make the outline square**

In [ ]:
def get_square_bounds(gdf, target_epsg=28992):
    """
    Computes a square bounding box centered on the original bounds of a GeoDataFrame.

    Parameters:
    - gdf: GeoDataFrame
    - target_epsg: EPSG code to reproject the GeoDataFrame before computing bounds (default: 28992)

    Returns:
    - square_bounds: NumPy array [minx, miny, maxx, maxy] of the square bounding box
    """
    # Project to desired CRS
    gdf_proj = gdf.to_crs(epsg=target_epsg)

    # Original bounds
    minx, miny, maxx, maxy = gdf_proj.total_bounds

    # Width and height
    width = maxx - minx
    height = maxy - miny

    # Max side length
    max_side = max(width, height)

    # Center coordinates
    center_x = (minx + maxx) / 2
    center_y = (miny + maxy) / 2

    # Square bounds
    half_side = max_side / 2
    square_bounds = np.array([
        center_x - half_side,
        center_y - half_side,
        center_x + half_side,
        center_y + half_side
    ])

    return square_bounds


In [ ]:
square_bounds_VEL = get_square_bounds(VELgdf)

### 1.2. Download WCS:
Download entire data **directly** if it is a coarse resolution, or **in tiles** (if finer resolution), followed by merging.

In [ ]:
# ── Cached Axis Label Fetcher ──────────────────────────────────────────────

_axis_cache = {}

def get_axis_labels(wcs_base: str, cov_id: str):
    """
    Fetch the canonical axis names for a given coverage from the WCS DescribeCoverage endpoint,
    using an in-memory cache to avoid repeated HTTP calls for the same coverage.
    """
    cache_key = (wcs_base, cov_id)
    if cache_key in _axis_cache:
        return _axis_cache[cache_key]

    params = {
        "service":    "WCS",
        "version":    "2.0.1",
        "request":    "DescribeCoverage",
        "coverageId": cov_id,
    }

    try:
        r = requests.get(wcs_base.replace("/ows", "/wcs"), params=params, timeout=15)
        r.raise_for_status()
        root = ET.fromstring(r.content)

        ns = {"gml": "http://www.opengis.net/gml/3.2"}
        env = root.find(".//gml:Envelope", namespaces=ns)
        labels = env.attrib["axisLabels"].split()
        _axis_cache[cache_key] = (labels[0], labels[1])
        return labels[0], labels[1]

    except Exception as e:
        print(f"⚠️ Failed to fetch axis labels for {cov_id}: {e}")
        # fallback to standard 'x', 'y' if failed
        return "x", "y"

def get_base_wfs_url(full_url):
    parsed = urlparse(full_url)
    return urlunparse((parsed.scheme, parsed.netloc, parsed.path, '', '', ''))

def detect_wcs_version(cap_url):
    def try_detect(url):
        try:
            r = requests.get(url, timeout=10)
            r.raise_for_status()
            content = r.content.decode("utf-8")
            root = ET.fromstring(content)

            # 1. Try version attribute
            ver = root.attrib.get("version")
            if ver in ("2.0.1", "1.1.1", "1.0.0"):
                return ver

            # 2. Fallback: search for known namespaces
            if "http://www.opengis.net/wcs/2.0" in content:
                return "2.0.1"
            if "http://www.opengis.net/wcs/1.1" in content:
                return "1.1.1"
            if "http://www.opengis.net/wcs" in content:
                return "1.0.0"
        except Exception as e:
            print(f"⚠️ Failed WCS version check for {url}: {e}")
        return None

    # 1. Try full URL first
    version = try_detect(cap_url)
    if version:
        return version

    # 2. Fallback: use cleaned base URL and add GetCapabilities
    base_url = get_base_wfs_url(cap_url)
    fallback_url = f"{base_url}?service=WCS&request=GetCapabilities"
    version = try_detect(fallback_url)
    if version:
        return version

    # 3. Default fallback
    return "1.0.0"

def split_square_tiles(outline, bbox, resx, resy, max_px=4000):
    """
    Split bbox = [xmin, ymin, xmax, ymax] into square tiles so that
    each tile is <= max_px × max_px pixels at resolution resx, resy.
    
    Only return tiles that intersect the given 'outline' geometry.
    'outline' can be a shapely Polygon/MultiPolygon or a GeoDataFrame.
    """
    # Ensure 'outline' is a shapely geometry
    if hasattr(outline, "geometry"):  # It's a GeoDataFrame
        outline_geom = outline.unary_union
    else:  # It's already a shapely geometry
        outline_geom = outline

    xmin, ymin, xmax, ymax = bbox
    tile_size = min(max_px * resx, max_px * resy)

    nx = math.ceil((xmax - xmin) / tile_size)
    ny = math.ceil((ymax - ymin) / tile_size)
    
    tiles = []
    for i in range(nx):
        for j in range(ny):
            x0 = xmin + i * tile_size
            y0 = ymin + j * tile_size
            x1 = min(x0 + tile_size, xmax)
            y1 = min(y0 + tile_size, ymax)
            
            tile_geom = box(x0, y0, x1, y1)
            
            if tile_geom.intersects(outline_geom):
                tiles.append([x0, y0, x1, y1])

    return tiles

def convert_bbox_to_degrees(bbox):
    transformer = Transformer.from_crs("EPSG:28992", "EPSG:4258", always_xy=True)
    xmin, ymin = transformer.transform(bbox[0], bbox[1])
    xmax, ymax = transformer.transform(bbox[2], bbox[3])
    return [xmin, ymin, xmax, ymax]

# ── Download function ──────────────────────────────────────────────────────

def download_tile_28992(row, bbox, resx, resy, tile_index):
    """
    Download one EPSG:28992 tile at resolution (resx,resy) passed in.
    """
    base_url = row["wcs_getcapabilities_url"].split("?")[0]
    version  = detect_wcs_version(row["wcs_getcapabilities_url"])

    if not version:
        print(f"❌ {row['layer']}: cannot detect WCS version")
        return None

    xmin, ymin, xmax, ymax = [round(v, 3) for v in bbox]
    width_px  = int((xmax - xmin) / resx)
    height_px = int((ymax - ymin) / resy)
    if width_px > 4000 or height_px > 4000:
        print(f"❌ {row['layer']}: {width_px}×{height_px} px > 4000 limit")
        return None

    layer    = row["layer"]
    out_path = os.path.join(tile_folder, f"{layer}_tile{tile_index}.tif")

    x_label, y_label = get_axis_labels(base_url, layer)
    
    # Common params for WCS 2.0.1
    common = {
        "service":       "WCS",
        "request":       "GetCoverage",
        "version":       version,
        "format":        "image/tiff",
        "coverageId":    layer,
        "subset":        [f"{x_label}({xmin},{xmax})", f"{y_label}({ymin},{ymax})"],
        "subsettingcrs":"EPSG:28992",
        "outputcrs":     "EPSG:28992"
    }

    # WCS 2.0.1: try scaleSize, then resx/resy
    if version == "2.0.1":
        # 1) scaleSize
        p = {**common,
            "scaleSize":   f"{x_label}({width_px})",
            "scaleSize_y": f"{y_label}({height_px})"}
        print("🔗", f"{base_url}?{urlencode(p, doseq=True)}")
        try:
            r = requests.get(base_url, params=p, timeout=60); r.raise_for_status()
            if "image" in r.headers.get("Content-Type","").lower():
                open(out_path,"wb").write(r.content)
                print(f"✅ {layer} tile {tile_index} (scaleSize)")
                return out_path
        except Exception as e:
            print("⚠️ scaleSize failed:", e)

        # 2) fallback to resx/resy
        p = {**common,
             "resx": round(resx,5),
             "resy": round(resy,5)}
        print("🔗", f"{base_url}?{urlencode(p, doseq=True)}")
        try:
            r = requests.get(base_url, params=p, timeout=60); r.raise_for_status()
            if "image" in r.headers.get("Content-Type","").lower():
                open(out_path,"wb").write(r.content)
                print(f"✅ {layer} tile {tile_index} (resx/resy)")
                return out_path
        except Exception as e:
            print("⚠️ resx/resy failed:", e)

    # WCS 1.0.0: BBOX + width/height
    else:
        p = {
            "service": "WCS", "request": "GetCoverage",
            "version": version, "coverage": layer,
            "CRS":    "EPSG:28992",
            "BBOX":   f"{xmin},{ymin},{xmax},{ymax}",
            "width":  width_px, "height": height_px,
            "format": "image/tiff"
        }
        print("🔗", f"{base_url}?{urlencode(p, doseq=True)}")
        try:
            r = requests.get(base_url, params=p, timeout=60); r.raise_for_status()
            if "image" in r.headers.get("Content-Type","").lower():
                open(out_path,"wb").write(r.content)
                print(f"✅ {layer} tile {tile_index} (WCS1.0)")
                return out_path
        except Exception as e:
            print("⚠️ WCS1.0 failed:", e)

    print(f"❌ {layer} tile {tile_index} failed (v{version})")
    return None

###### DEGREES

def download_tile_degrees(row, bbox_m, native_resolution, tile_folder, output_folder):
    """
    Downloads WCS coverages in geographic CRS (EPSG:4258/4326).
    - bbox_m: your EPSG:28992 bbox in meters
    - native_resolution: bool
    Returns list of saved TIFF file paths.
    """
    from pyproj import Transformer
    import ast, os, requests
    from urllib.parse import urlencode

    # 1) Convert bbox from 28992→degrees
    tf = Transformer.from_crs("EPSG:28992", f"EPSG:{row['crs_epsg']}", always_xy=True)
    xmin_d, ymin_d = tf.transform(bbox_m[0], bbox_m[1])
    xmax_d, ymax_d = tf.transform(bbox_m[2], bbox_m[3])
    bbox_deg = [xmin_d, ymin_d, xmax_d, ymax_d]

    # 2) Parse native spacing (in degrees) from CSV
    native_resx, native_resy = map(float, ast.literal_eval(row["spatial_resolution"]))

    # 3) Decide deg/px and tile grid
    if native_resolution:
        # true native spacing → split into 4000×4000-px squares
        resx, resy = native_resx, native_resy
        tiles = split_square_tiles(bbox_deg, resx, resy, max_px=4000)
    else:
        # downsample whole extent to one tile ≤4000 px on longest side
        w = xmax_d - xmin_d
        h = ymax_d - ymin_d
        target_px = 4000
        # choose degree/px so that max(w, h)/deg_per_px = ≤4000
        deg_per_px = max(w, h) / target_px
        resx = resy = deg_per_px
        tiles = [bbox_deg]

    saved = []
    base    = row["wcs_getcapabilities_url"].split("?")[0]
    version = detect_wcs_version(row["wcs_getcapabilities_url"])
    if not version:
        print(f"❌ {row['layer']}: cannot detect WCS version")
        return saved

    coverage = row["layer"]
    crs_code = int(row["crs_epsg"])

    for idx, tb in enumerate(tiles, start=1):
        xmin, ymin, xmax, ymax = [round(v, 9) for v in tb]
        wp = int((xmax - xmin) / resx)
        hp = int((ymax - ymin) / resy)
        if wp > 4000 or hp > 4000:
            print(f"❌ {coverage}: {wp}×{hp} px > 4000")
            continue

        if native_resolution:
            out = os.path.join(tile_folder, f"{coverage}_deg_tile{idx}.tif")
        else:
            out = os.path.join(tile_folder, f"{coverage}.tif")
        
        x_label, y_label = get_axis_labels(base, coverage)
        
        common = {
            "service":       "WCS",
            "request":       "GetCoverage",
            "version":       version,
            "format":        "image/tiff",
            "coverageId":    coverage,
            "subset": [f"{x_label}({xmin},{xmax})", f"{y_label}({ymin},{ymax})"],
            "subsettingcrs": f"EPSG:{crs_code}",
            "outputcrs":     f"EPSG:{crs_code}"
        }

        # WCS 2.x
        if version.startswith("2"):
            # try scaleSize
            p = {**common, "scaleSize":   f"{x_label}({wp})", "scaleSize_y": f"{y_label}({hp})"}
            print("🔗", f"{base}?{urlencode(p, doseq=True)}")
            try:
                r = requests.get(base, params=p, timeout=60); r.raise_for_status()
                if "image" in r.headers.get("Content-Type", "").lower():
                    open(out, "wb").write(r.content)
                    print(f"✅ {coverage} tile{idx} (scaleSize)")
                    saved.append(out)
                    continue
            except Exception as e:
                print("⚠️ scaleSize failed:", e)

            # fallback to resx/resy
            p = {**common, "resx": round(resx, 9), "resy": round(resy, 9)}
            print("🔗", f"{base}?{urlencode(p, doseq=True)}")
            try:
                r = requests.get(base, params=p, timeout=60); r.raise_for_status()
                if "image" in r.headers.get("Content-Type", "").lower():
                    open(out, "wb").write(r.content)
                    print(f"✅ {coverage} tile{idx} (resx/resy)")
                    saved.append(out)
                    continue
            except Exception as e:
                print("⚠️ resx/resy failed:", e)

        # WCS 1.0
        else:
            p = {
                "service": "WCS",
                "request": "GetCoverage",
                "version": version,
                "coverage": coverage,
                "CRS":      f"EPSG:{crs_code}",
                "BBOX":     f"{xmin},{ymin},{xmax},{ymax}",
                "width":    wp, "height": hp,
                "format":   "image/tiff"
            }
            print("🔗", f"{base}?{urlencode(p, doseq=True)}")
            try:
                r = requests.get(base, params=p, timeout=60); r.raise_for_status()
                if "image" in r.headers.get("Content-Type", "").lower():
                    open(out, "wb").write(r.content)
                    print(f"✅ {coverage} tile{idx} in {len(tiles)} tile(s)")
                    saved.append(out)
                    continue
            except Exception as e:
                print("WCS1.0 failed:", e)

        print(f"{coverage} tile{idx} failed (v{version})")

    return saved

def merge_tiles_with_gdalwarp(layer, tile_folder, output_folder):
    tile_pattern = os.path.join(tile_folder, f"{layer}*.tif")
    tile_paths = glob.glob(tile_pattern)

    if not tile_paths:
        raise FileNotFoundError(f"No tiles found matching pattern: {tile_pattern}")

    print(f"✅ Found {len(tile_paths)} tiles to merge using gdal.Warp()...")

    output_path = os.path.join(output_folder, f"{layer}.tif")

    # Debug: check first tile
    test_ds = gdal.Open(tile_paths[0])
    if test_ds is None:
        raise RuntimeError(f"Could not open sample tile: {tile_paths[0]}")

    if not output_path.lower().endswith(".tif"):
        raise ValueError(f"Output path must be a .tif file, got: {output_path}")

    print(f"Merging into: {output_path}")

    # Merge tiles
    vrt = gdal.Warp(output_path, tile_paths, format='GTiff', options=['COMPRESS=DEFLATE'])
    if vrt is None:
        raise RuntimeError("GDAL merge failed. Check input files and output path.")
    vrt = None  # Close dataset

    print(f"✅ Merge completed: {output_path}")

    # Delete the original tiles after merging
    for tile in tile_paths:
        try:
            os.remove(tile)
            #print(f"Deleted tile: {tile}")
        except Exception as e:
            print(f"⚠️ Could not delete tile {tile}: {e}")


**Executing WCS data download workflow**

In [ ]:
# === USER INPUTS ===

df = pd.read_csv("checkpoint05_ngr_WCS_metadata.csv")
#df['download']='yes'

outline_28992 = veluwe28992
bbox_28992 = square_bounds_VEL

native_resolution = True   
tile_folder = "wcs_tiles"
output_folder = "wcs_outputs"

os.makedirs(tile_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)


In [ ]:
# ──────────────────────────────────────────────────────────────────────────────
# 0.  Prepare a list that will hold one log-entry per layer
# ──────────────────────────────────────────────────────────────────────────────

log_records = []        # → [{"layer":..., "success":..., "tiles":..., "duration_s":...}, ...]

# === MAIN LOOP ===

for _, row in df[df["download"] == "yes"].iterrows():
    crs   = int(row["crs_epsg"])
    layer = row["layer"]

    print(f"\n📦 Processing {layer} (EPSG:{crs})")

    t0 = time.perf_counter()          # ── start timer
    downloaded_tiles = []             # track paths of downloaded tiles

    try:
        if crs == 28992:
            native_resx, native_resy = map(float, ast.literal_eval(row["spatial_resolution"]))

            if native_resolution:
                resx, resy = native_resx, native_resy
                tiles = split_square_tiles(outline_28992, bbox_28992, resx, resy)
                print(f"   → Downloading in {len(tiles)} tile(s) at {resx:.6f}×{resy:.6f} m/px")
                for idx, tb in enumerate(tqdm(tiles, desc="   📥 tiles"), start=1):
                    path = download_tile_28992(row, tb, resx, resy, idx)
                    if path:
                        downloaded_tiles.append(path)
            else:
                w = bbox_28992[2] - bbox_28992[0]
                h = bbox_28992[3] - bbox_28992[1]
                resx, resy = w / 4000, h / 4000
                print(f"   → Downloading 1 tile at {resx:.6f}×{resy:.6f} m/px")
                path = download_tile_28992(row, bbox_28992, resx, resy, 1)
                if path:
                    downloaded_tiles.append(path)

        elif crs in (4258, 4326):
            print(f"   → Downloading geographic coverage with native_resolution={native_resolution}")

            downloaded_tiles = download_tile_degrees(
                row, bbox_28992, native_resolution, tile_folder, output_folder
            )

        else:
            print(f"❌ Unsupported CRS EPSG:{crs}, skipping.")
            success = False
            num_tiles = 0
            raise RuntimeError("Unsupported CRS")

        # ── Merge if we got at least one tile ───────────────────────────────
        if downloaded_tiles:
            merge_tiles_with_gdalwarp(layer, tile_folder, output_folder)
        else:
            print(f"⚠️ No tiles downloaded for {layer}, skipping merge.")

        success   = bool(downloaded_tiles)
        num_tiles = len(downloaded_tiles)

    except Exception as e:
        print(f"🚨 Error while processing {layer}: {e}")
        success   = False
        num_tiles = 0

    # ── stop timer & log result ──────────────────────────────────────────────
    duration = round(time.perf_counter() - t0, 2)     # seconds with 0.01 precision
    log_records.append(
        {"layer": layer, "success": success, "tiles": num_tiles, "duration_s": duration}
    )

# ──────────────────────────────────────────────────────────────────────────────
# 4.  Convert log to DataFrame and save
# ──────────────────────────────────────────────────────────────────────────────
wcs_download_log = pd.DataFrame(log_records)
csv_path = Path(output_folder) / "wcs_download_log.csv"
wcs_download_log.to_csv(csv_path, index=False)

print(f"\n✅ Written summary to: {csv_path}")
print(wcs_download_log.head())


---

### 1.3. Download WFS:

In [ ]:
def get_base_wfs_url(full_url):
    parsed = urlparse(full_url)
    return urlunparse((parsed.scheme, parsed.netloc, parsed.path, '', '', ''))

def get_wfs_feature_count(wfs_base_url, layer_name, verbose=False):
    wfs_base_url = get_base_wfs_url(wfs_base_url)

    def build_url(version):
        if version == "2.0.0":
            params = {
                'service': 'WFS',
                'version': version,
                'request': 'GetFeature',
                'typenames': layer_name,
                'resultType': 'hits'
            }
        else:  # version 1.1.0
            params = {
                'service': 'WFS',
                'version': version,
                'request': 'GetFeature',
                'typeName': layer_name,
                'resultType': 'hits'
            }
        return f"{wfs_base_url}?{urlencode(params)}"

    for version in ["1.1.0", "2.0.0"]:
        url = build_url(version)
        for attempt in range(1, 6):
            try:
                if verbose:
                    print(f"Attempt {attempt} with WFS {version}...")
                r = requests.get(url, timeout=30)
                r.raise_for_status()
                tree = etree.fromstring(r.content)

                if version == "2.0.0":
                    count = tree.attrib.get("numberMatched")
                else:
                    count = tree.attrib.get("numberOfFeatures")

                return int(count) if count is not None else 0

            except Exception as e:
                if verbose:
                    print(f"Attempt {attempt} failed with WFS {version}: {e}")
                if attempt < 5:
                    time.sleep(2)

    return None

# === DOWNLOAD FUNCTION ===

def download_full_wfs_layer_as_geodataframe(
    wfs_base_url,
    layer_name,
    bbox=None,
    bbox_crs=None,
    max_features_per_request=2000,
    get_feature_count_func=None,
    verbose=False
):
    if get_feature_count_func is None:
        raise ValueError("You must provide get_feature_count_func to retrieve total number of features.")

    total_features = get_feature_count_func(wfs_base_url, layer_name, verbose=verbose)
    if total_features is None:
        raise RuntimeError("Could not retrieve total feature count. Aborting.")

    if verbose:
        print(f"Total features to download: {total_features}")

    batch_gdfs = []

    for start_index in range(0, total_features, max_features_per_request):
        if verbose:
            print(f"Downloading features {start_index} to {start_index + max_features_per_request}...")

        gdf_batch = download_wfs_layer_as_geodataframe(
            wfs_base_url=wfs_base_url,
            layer_name=layer_name,
            bbox=bbox,
            bbox_crs=bbox_crs,
            max_features=max_features_per_request,
            start_index=start_index
        )

        if gdf_batch is None or len(gdf_batch) == 0:
            if verbose:
                print("No more features returned, stopping.")
            break

        batch_gdfs.append(gdf_batch)

    if len(batch_gdfs) == 0:
        if verbose:
            print("No features downloaded.")
        return None

    gdf_full = pd.concat(batch_gdfs, ignore_index=True)
    if verbose:
        print(f"Downloaded total {len(gdf_full)} features.")

    return gdf_full


def download_wfs_layer_as_geodataframe(
    wfs_base_url, 
    layer_name, 
    bbox=None, 
    bbox_crs='EPSG:28992', 
    max_features=1000, 
    start_index=0,
    verbose=False
):
    import tempfile
    from urllib.parse import urlencode

    def build_params(version):
        if version == '2.0.0':
            key_layer = 'typenames'
            key_count = 'count'
        else:
            key_layer = 'typeName'
            key_count = 'maxFeatures'

        params = {
            'service': 'WFS',
            'version': version,
            'request': 'GetFeature',
            key_layer: layer_name,
            'outputFormat': 'application/json',
            key_count: max_features,
            'startIndex': start_index
        }

        if bbox is not None:
            bbox_str = ','.join(map(str, bbox)) + f',{bbox_crs}'
            params['bbox'] = bbox_str

        return params

    for version in ['1.1.0', '2.0.0']:
        params = build_params(version)
        url = f"{wfs_base_url}?{urlencode(params)}"

        print(f"\n🌍 Trying WFS version {version}")
        print(f"🔗 URL: {url}")

        try:
            r = requests.get(url, timeout=60)
            print(f"📥 HTTP status: {r.status_code}")
            print(f"📄 First 300 characters of response:\n{r.text[:300]}")

            r.raise_for_status()

            with tempfile.NamedTemporaryFile(suffix=".geojson", delete=False) as tmpfile:
                tmp_filename = tmpfile.name
                tmpfile.write(r.content)

            gdf = gpd.read_file(tmp_filename)
            os.remove(tmp_filename)

            if gdf is not None and len(gdf) > 0:
                print(f"✅ Retrieved {len(gdf)} features using WFS {version}")
                return gdf
            else:
                print(f"⚠️ No data returned for version {version}. Trying fallback...")

        except Exception as e:
            print(f"❌ Error with WFS version {version}: {e}")

    print("❌ All versions failed or returned no features.")
    return None

def sanitize_datetime_columns(gdf):
    """
    Converts all datetime-like columns in a GeoDataFrame to ISO-formatted strings.

    This is necessary because some backends (e.g. Fiona/GeoJSON writer) 
    do not support pandas Timestamp or timezone-aware datetime objects.
    """
    for col in gdf.columns:
        if pd.api.types.is_datetime64_any_dtype(gdf[col]) or isinstance(gdf[col].dtype, pd.DatetimeTZDtype):
            gdf[col] = gdf[col].dt.tz_localize(None).dt.strftime('%Y-%m-%dT%H:%M:%S')
        elif gdf[col].dtype == 'object':
            if gdf[col].apply(lambda x: isinstance(x, pd.Timestamp)).any():
                gdf[col] = gdf[col].apply(
                    lambda x: x.tz_localize(None).strftime('%Y-%m-%dT%H:%M:%S') if isinstance(x, pd.Timestamp) else x
                )
    return gdf

In [ ]:
# === USER INPUTS ===

df = pd.read_csv("checkpoint05_ngr_WFS_metadata.csv")
df['download']='yes'
bbox_28992 = [156913.384, 439295.44, 221547.716, 503929.772]
output_folder = "wfs_outputs"
os.makedirs(output_folder, exist_ok=True)


In [ ]:
#df[df["download"]=="yes"]

**Executing WFS data download workflow**

In [ ]:
# ------------------------------------------------------------------
# 1.  Prepare a list that will collect one record per row processed
# ------------------------------------------------------------------
download_log = []  # each element: {"layer": ..., "status": ..., "duration_s": ...}

# ------------------------------------------------------------------
# 2.  Main download loop with retry and timing
# ------------------------------------------------------------------
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Downloading WFS layers"):
    layer_name = row.get("layer", "unknown_layer").strip()
    wanted = str(row.get("download", "")).strip().lower() == "yes"

    if not wanted:
        download_log.append({"layer": layer_name, "status": "skipped", "duration_s": 0})
        continue

    status = "failed"
    total_start = time.perf_counter()

    for attempt in range(1, 4):  # Try up to 3 times
        print(f"\n🔁 Attempt {attempt}/3 for layer: {layer_name}")
        try:
            wfs_base_url = get_base_wfs_url(row["wfs_getcapabilities_url"])

            gdf = download_full_wfs_layer_as_geodataframe(
                wfs_base_url=wfs_base_url,
                layer_name=layer_name,
                bbox=bbox_28992,
                bbox_crs="EPSG:28992",
                max_features_per_request=1000,
                get_feature_count_func=get_wfs_feature_count,
                verbose=True
            )

            if gdf is not None:
                if 'fid' in gdf.columns:
                    print(f"Dropping 'fid' column before saving: {layer_name}")
                    gdf = gdf.drop(columns='fid')

                gdf = sanitize_datetime_columns(gdf)

                output_path = os.path.join(output_folder, f"{layer_name.replace(':', '_')}.geojson")

                datetime_cols = [
                    col for col in gdf.columns
                    if pd.api.types.is_datetime64_any_dtype(gdf[col]) or isinstance(gdf[col].dtype, pd.DatetimeTZDtype)
                ]
                if datetime_cols:
                    print(f"⚠️ Unconverted datetime columns before saving {layer_name}: {datetime_cols}")

                gdf.to_file(output_path, driver="GeoJSON")
                status = "success"
                break  # Exit the retry loop if successful
            else:
                status = "empty-result"

        except Exception as e:
            status = f"failed: {e}"
            print(f"❌ Attempt {attempt} failed for layer '{layer_name}': {e}")
            time.sleep(2)  # Wait before next attempt

    duration = time.perf_counter() - total_start
    download_log.append({"layer": layer_name, "status": status, "duration_s": round(duration, 2)})
    time.sleep(2)  # Wait before moving to the next layer

# ------------------------------------------------------------------
# 3.  Write the log to CSV
# ------------------------------------------------------------------
log_df = pd.DataFrame(download_log)
log_path = os.path.join(output_folder, "download_summary.csv")
log_df.to_csv(log_path, index=False)

print(f"\n✔️  Download summary written to: {log_path}")


---